 <div>
<img src="https://edlitera-images.s3.amazonaws.com/new_edlitera_logo.png" width="500"/>
</div>

# `NLP libraries based on neural networks`

* two important libraries:
    <br>
    
    * Gensim - best known for the ability to create high quality word vectors
    
    * spaCy - best known for the quality of its text preprocessing techniques

* both use neural networks to perform various tasks

# `Gensim`

* open source Python library for NLP

* designed to process raw data ("plain text") very efficiently using unsupervised Machine Learning algorithms

**Characteristics**

* industry tested algorithms (less focus on very recent research)


* high performance (all implementations of algorithms are as optimized as possible)


* offers memory independence
    

**We can use Gensim for:**

* creating word vectors
* creating document vectors
* creating corpora
* comparing document
* topic modeling
* etc.

**Some of the algorithms:**
    
* Word2Vec
* FastText
* Latent Semantic Indexing
* Latent Dirichlet Allocation

**We will focus on Word2Vec: it is the most famous algorithm for producing high-quality word embeddings**

# `Word2Vec`

* set of neural arhitectures designed to calculate continuous representations of words for large datasets
    * "window based models"
    * allows us to capture complex patterns

* easy way to create word embeddings for large datasets

* when we previously talked about word embeddings we gave an example with 3 features
    * in practice we use values on the order of hundres
    
    
* this is why cosine similarity works well: it works with any number of dimensions

### `Neural arhitectures`

* **CBOW (Continuous Bag Of Words)**
* **Skip-gram**

## `CBOW`


<center><img src="https://edlitera-images.s3.amazonaws.com/CBOW.png" width="900">

inspired by:
<br>

https://www.r-craft.org/r-news/get-busy-with-word-embeddings-an-introduction/

* our goal is to predict the centre word based on some context words

* by including words from both sides we make our predictions a lot more precise


* the number of words included as context words from both sides is called the **context window**
    * the context windows size usually set around 5


* order of words is NOT important since we average everything out


<center><img src="https://edlitera-images.s3.amazonaws.com/CBOW_model.png" width="1000">

source:
<br>

    
https://techblog.ezra.com/different-embedding-models-7874197dc410

## `Skip-gram`


<center><img src="https://edlitera-images.s3.amazonaws.com/Skip_gram.png" width="700">

source:
<br>

https://www.r-craft.org/r-news/get-busy-with-word-embeddings-an-introduction/

* our goal is to predict the context words that surround some centre word 

* the context windows size usually set around 10

* works relatively well for smaller datasets, but is otherwise really slow to train
    * training vanilla skip-gram on big datasets is not feasible


<center><img src="https://edlitera-images.s3.amazonaws.com/Skip_gram_model.png" width="1000">

inspired by:
<br>

    
    
https://techblog.ezra.com/different-embedding-models-7874197dc410

# `Text preprocessing for Word2Vec`

* so far, we have performed:

    * punctuations and stop words removal
    * stemming or lemmatization 

* **BUT: that won't always lead to the best results when we create Word2Vec model**

### Why Word2Vec sometimes works better with less preprocessing?

* generally, Word2Vec learns word embeddings using context (it learns by looking at what words are near to the word we are looking for)
    <br>

* because context is important, sometimes including a lot of preprocessing is not ideal

### When do we preprocess text data for Word2Vec?

* there is no definitive answer, since it depends on the situation (on the task we are trying to solve )

* there are some cases where some preprocessing steps are very detrimental to the overall performance of a model, BUT THERE ARE ALSO CASES WHERE PREPROCESSING IMPROVES ACCURACY !

# `Gensim Word2Vec example`

### Loading data

In [173]:
# Import necessary libraries

import nltk                       
import pandas as pd               
import re                         
from nltk.corpus import stopwords  
from gensim import parsing        # For data preprocessing in Gensim
import gensim
from gensim.models import Word2Vec
import numpy as np

In [174]:
# Import data and create a DataFrame
# Take a look at the first 5 rows

wine_data = pd.read_csv("https://edlitera-datasets.s3.amazonaws.com/wine_data_classification.csv")

wine_data.head()

,description,wine_type
0,"Aromas include tropical fruit, broom, brimston...",great_wine
1,"This is ripe and fruity, a wine that is smooth...",great_wine
2,"Tart and snappy, the flavors of lime flesh and...",great_wine
3,"Pineapple rind, lemon pith and orange blossom ...",great_wine
4,"Much like the regular bottling from 2012, this...",great_wine


### Preprocessing

In [175]:
# Import RegEx

import re

# Create a function that deals with contractions
# Very useful snippet of code

def decontracted(phrase):
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# Deal with contractions

wine_data["description"] = wine_data["description"].apply(decontracted)

# Remove the "wine_type" column 

wine_data_without_labels = wine_data.drop(columns=["wine_type"])

In [176]:
# Take a look at cleaned data

wine_data_without_labels

,description
0,"Aromas include tropical fruit, broom, brimston..."
1,"This is ripe and fruity, a wine that is smooth..."
2,"Tart and snappy, the flavors of lime flesh and..."
3,"Pineapple rind, lemon pith and orange blossom ..."
4,"Much like the regular bottling from 2012, this..."
...,...
129966,Notes of honeysuckle and cantaloupe sweeten th...
129967,Citation is given as much as a decade of bottl...
129968,Well-drained gravel soil gives this wine its c...
129969,"A dry style of Pinot Gris, this is crisp with ..."


In [177]:
# Create list of lists
# every sublist contains a tokenized sentence

sentences = [row.split(".") for row in wine_data_without_labels["description"]]
print(sentences[0][0].split())

sentences_tokenized = [sublist[0].split() for sublist in sentences]

['Aromas', 'include', 'tropical', 'fruit,', 'broom,', 'brimstone', 'and', 'dried', 'herb']


In [178]:
sentences_tokenized

[['Aromas',
  'include',
  'tropical',
  'fruit,',
  'broom,',
  'brimstone',
  'and',
  'dried',
  'herb'],
 ['This',
  'is',
  'ripe',
  'and',
  'fruity,',
  'a',
  'wine',
  'that',
  'is',
  'smooth',
  'while',
  'still',
  'structured'],
 ['Tart',
  'and',
  'snappy,',
  'the',
  'flavors',
  'of',
  'lime',
  'flesh',
  'and',
  'rind',
  'dominate'],
 ['Pineapple',
  'rind,',
  'lemon',
  'pith',
  'and',
  'orange',
  'blossom',
  'start',
  'off',
  'the',
  'aromas'],
 ['Much',
  'like',
  'the',
  'regular',
  'bottling',
  'from',
  '2012,',
  'this',
  'comes',
  'across',
  'as',
  'rather',
  'rough',
  'and',
  'tannic,',
  'with',
  'rustic,',
  'earthy,',
  'herbal',
  'characteristics'],
 ['Blackberry',
  'and',
  'raspberry',
  'aromas',
  'show',
  'a',
  'typical',
  'Navarran',
  'whiff',
  'of',
  'green',
  'herbs',
  'and,',
  'in',
  'this',
  'case,',
  'horseradish'],
 ['Here',
  'is',
  'a',
  'bright,',
  'informal',
  'red',
  'that',
  'opens',
  'wit

### Training

In [179]:
# Train CBOW model

#CBOW_model = Word2Vec(sentences_tokenized, min_count=3, size=100, window=3, sg=0)
CBOW_model = Word2Vec(sentences_tokenized, min_count=3, vector_size=100, window=3, sg=0)

**Note:** depending on the version of Python you are using, you might need to replace the "size" argument with the "vector size" argument: they do the same thing and both define how many dimensions you want your vectors to have, vector_size is just the wording for the argument in the latest version of Gensim

In [180]:
CBOW_model.wv["lime"]

array([-0.5794264 ,  0.8548626 ,  0.09236417, -2.6009402 ,  2.0056615 ,
        0.78707784, -1.2063197 ,  0.6124262 , -2.0209816 ,  0.49088135,
       -1.919563  , -0.22505675, -0.8595851 , -1.7703209 ,  1.1694493 ,
        1.492688  , -0.10026009,  0.834901  ,  0.8090598 , -0.14939818,
        0.45437756,  1.2883111 , -1.1642649 , -0.54008734, -0.6300704 ,
       -1.1545609 , -0.5387825 , -0.29313415,  0.32404128, -0.02980537,
       -1.2895707 ,  2.4803257 ,  0.55579996,  0.07701989, -1.5664504 ,
        1.4369888 ,  0.6243627 , -0.5745813 , -1.3806939 , -0.30914766,
        1.0354164 ,  0.83058935,  1.1285067 , -0.49803877,  0.46346956,
        1.2684065 , -0.1949703 , -0.5220325 ,  1.3265295 , -0.7130907 ,
       -1.1017516 ,  0.22667389, -1.6713036 , -0.2272086 ,  1.3170631 ,
       -0.9302475 ,  1.5045874 ,  2.594617  , -0.7898458 ,  0.71149284,
        0.75834376,  1.7763054 , -0.1712613 ,  0.12062736,  0.77472097,
       -0.3282693 , -0.08101093, -0.4679528 , -1.3231887 ,  0.14

In [181]:
CBOW_model.wv.most_similar("try")[:5]

[('understand', 0.8945558071136475),
 ('watch', 0.8934782147407532),
 ('imagine', 0.8876318335533142),
 ('trying', 0.8822362422943115),
 ('celebrate', 0.880940318107605)]

In [182]:
CBOW_model.wv.similarity("bad","badly")

0.6188893

In [183]:
# Take a look at the top 5 most similar words

CBOW_model.wv.most_similar("fruity")[:5]

[('fruity,', 0.7265409827232361),
 ('approachable', 0.6875606775283813),
 ('textured', 0.6738972663879395),
 ('delicious', 0.664882242679596),
 ('juicy', 0.6590139865875244)]

In [184]:
CBOW_model.wv.similarity('tangerine', 'grapefruit')

0.85324687

# `Gensim Word2Vec` take home exercise

**Create a `Skip-gram model` using `Gensim` and train it on the data inside the `Review Text` column of the dataset stored inside `https://edlitera-datasets.s3.amazonaws.com/womens_clothing_reviews.csv` file. After training take a look at what are the most similar words to the word `"shirt"` using that model.**

**Note: train the model both with, and without preprocessing your text data (removing stop words etc.) and compare the results.**

## `Solution:`

In [26]:
df = pd.read_csv('https://edlitera-datasets.s3.amazonaws.com/womens_clothing_reviews.csv')

In [29]:
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [40]:
df['Review Text'] = [x.split('.') for x in df['Review Text']]

In [43]:
df['Review Text'] = [x[0].split() for x in df['Review Text']]

In [44]:
df['Review Text']

0        [I, had, such, high, hopes, for, this, dress, ...
1                  [I, love,, love,, love, this, jumpsuit]
2        [This, shirt, is, very, flattering, to, all, d...
3        [I, love, tracy, reese, dresses,, but, this, o...
4        [I, aded, this, in, my, basket, at, hte, last,...
                               ...                        
19657    [I, was, very, happy, to, snag, this, dress, a...
19658            [It, reminds, me, of, maternity, clothes]
19659    [This, fit, well,, but, the, top, was, very, s...
19660    [I, bought, this, dress, for, a, wedding, i, h...
19661    [This, dress, in, a, lovely, platinum, is, fem...
Name: Review Text, Length: 19662, dtype: object

In [47]:
CBOW_model = Word2Vec(df['Review Text'],min_count = 3, vector_size = 100 , window = 3 , sg =0)

In [57]:
CBOW_model.wv.most_similar('shirt')[:5]

[('top', 0.9826173782348633),
 ('skirt', 0.9720269441604614),
 ('sweater', 0.9587072730064392),
 ('blouse', 0.956330418586731),
 ('dress', 0.9265361428260803)]

In [48]:
CBOW_model()

In [33]:
import re

def decontanact(phrase):
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

df['Review Text'] = df['Review Text'].apply(decontanact)

# `GloVe (Global and Vector)`

* unsupervised learning algorithm

* combination of count-based methods and predictive methods

* relies not only on local statistics, but also incorporates global statistics
    * Word2Vec relies on local statistics only

* we won't go in-depth about exactly how it works, because that requires going through and explaining a lot of complex mathematical equations
    * for those who want to go through the mathematical equations: https://nlp.stanford.edu/pubs/glove.pdf

* WHY WE CARE: **pretrained GloVe embeddings will typically outperform pretrained Word2Vec embeddings**
    * not always 
    * Word2Vec is much simpler, so it's easy to find domain specific Word2Vec embeddings; try and find some that will suit your needs

# `Using pretrained word embeddings`

* in the majority of cases, pre-trained embeddings will be better then those you can get by training on your data
    * **one exception: if you generate enormous amounts of your own data and it is very domain specific training your own model might be a better idea**

**Why are pre-trained models usually better**

* they are trained on gigantic corpora of very diverse words


* in most cases your corpus won't be big enough to get embeddings that are of the same or higher quality as the pretrained ones

**NOTE:** 


* always a good idea to first try out pre-trained embeddings and look at the results
    * at least you'll establish a baseline performance that you must beat with your own model


## `Using pretrained embeddings with Keras`

In [185]:
# Import necessary libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dense, LSTM, Dropout
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.metrics import BinaryAccuracy

In [186]:
# Get rid of randomness (as much as possible)

from numpy.random import seed
seed(42)
import tensorflow
#tensorflow.random.set_random_seed(42)
tensorflow.random.set_seed(42)

In [187]:
# Take a sample from the wine data dataset
# Shuffle the sample data

df = wine_data.copy()

In [188]:
df.head()

,description,wine_type
0,"Aromas include tropical fruit, broom, brimston...",great_wine
1,"This is ripe and fruity, a wine that is smooth...",great_wine
2,"Tart and snappy, the flavors of lime flesh and...",great_wine
3,"Pineapple rind, lemon pith and orange blossom ...",great_wine
4,"Much like the regular bottling from 2012, this...",great_wine


In [189]:
# Map our label to 0 and 1

label_mapping = {
           'great_wine': 0,
           'superior_wine': 1,
           }


df['wine_type'] = df['wine_type'].map(label_mapping)

df

,description,wine_type
0,"Aromas include tropical fruit, broom, brimston...",0
1,"This is ripe and fruity, a wine that is smooth...",0
2,"Tart and snappy, the flavors of lime flesh and...",0
3,"Pineapple rind, lemon pith and orange blossom ...",0
4,"Much like the regular bottling from 2012, this...",0
...,...,...
129966,Notes of honeysuckle and cantaloupe sweeten th...,0
129967,Citation is given as much as a decade of bottl...,0
129968,Well-drained gravel soil gives this wine its c...,0
129969,"A dry style of Pinot Gris, this is crisp with ...",0


In [190]:
# Class count

count_class_0, count_class_1 = df.wine_type.value_counts()

print(count_class_0)
print(count_class_1)

96336
33635


In [191]:
# Divide by class

df_class_0 = df[df['wine_type'] == 0]
df_class_1 = df[df['wine_type'] == 1]

df_class_0_under = df_class_0.sample(count_class_1)
df = pd.concat([df_class_0_under, df_class_1], axis=0)

In [192]:
df = df.sample(frac=1).reset_index(drop=True)
df

,description,wine_type
0,This wine is broad on the palate and creamy in...,0
1,A classic Portuguese blend of Touriga Nacional...,0
2,"Aromas of cherry, spiced meat and plum start o...",0
3,"This full-bodied, concentrated wine shows plen...",1
4,This beautiful Riserva Brunello is characteriz...,1
...,...,...
67265,It is unheard of for a winery to hold back a S...,0
67266,"A nose of raspberry, brambles and confectioner...",0
67267,"Mulberry, cherry, iron, raw beef and roasted p...",1
67268,"Ripe and rounded, this softly textured wine ha...",1


In [193]:
# Separate dependent feature from the independent feature

X = df["description"]



In [194]:
X = df["description"]

y = df["wine_type"]

In [195]:
# Separate data into training data and testing data

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

In [196]:
# Separate data into training data and validation data

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.20, random_state=42
)

In [197]:
# Define tokenizer
# Set number of words
# Set value of oov token
# Leave everything else on default values

tokenizer = Tokenizer(
    num_words=20_000, 
    oov_token = "<OOV>")

In [198]:
# Fit tokenizer on train data

tokenizer.fit_on_texts(X_train)

In [199]:
#print("Count of characters:",tokenizer.word_counts)
#print("Length of text:",tokenizer.document_count)
#print("Character index",tokenizer.word_index)
print("Frequency of characters:",tokenizer.word_docs)

Frequency of characters: defaultdict(<class 'int'>, {'of': 33846, 'lives': 27, 'with': 28624, 'its': 6057, 'overtones': 133, 'shot': 124, 'rounded': 928, 'fine': 2823, 'here—the': 1, 'tba': 18, 'through': 4513, 'honey': 1395, 'up': 2506, 'this': 33534, 'all': 2625, 'it': 22258, 'acidity': 11340, 'viscosity': 35, 'musk': 15, 'and': 42261, 'hints': 1684, 'the': 35811, 'almost': 1387, 'sweetness': 803, 'dark': 4547, 'is': 31222, 'bitter': 899, 'billing': 8, 'to': 15805, 'fully': 447, 'enormously': 98, 'wine': 22333, 'sheer': 148, 'drink': 10852, 'now': 5205, 'granola': 14, 'that': 12386, 'a': 34366, 'nuts': 162, 'power': 1100, 'flavors': 19053, 'in': 16862, 'cabernet': 3294, 'blueberries': 184, 'roasted': 860, 'modern': 291, 'mocha': 1141, 'impresses': 112, 'flashy': 124, 'suggest': 380, 'fruity': 2589, 'tannins': 10703, 'acids': 626, 'soft': 3917, 'sweetened': 34, 'for': 7191, 'made': 2291, 'blackberries': 876, 'style': 2096, 'candy': 366, 'color': 1326, '2015': 413, 'enlivened': 27, 'ar

In [200]:
X_train

64683    It is all here—the viscosity, the rounded swee...
60496    Made in the modern style, this Cabernet is dar...
714      A rosé of 100% Mourvèdre, this bears mild flav...
7562     Here is a surprise from Mt. Etna that blew us ...
37918    This is undeniably an oaky wine with a promine...
                               ...                        
59820    Alluring white-pepper spice, with nuances of d...
58032    This is Fred Loimer is star wine, the summit o...
55848    A wonderful nose mixes pineapple/tropical frui...
37882    Grenache (52%) takes the lead on this wine fol...
52334    Leather, menthol, dark-skinned berry and light...
Name: description, Length: 43052, dtype: object

In [201]:
tokenizer.texts_to_sequences(X_train)

[[9,
  6,
  91,
  14098,
  2925,
  3,
  285,
  343,
  3,
  1330,
  5,
  4525,
  2,
  3,
  159,
  5,
  48,
  188,
  294,
  185,
  1387,
  51,
  7,
  83,
  20,
  8,
  4115,
  514,
  3364,
  98,
  13,
  35,
  6122],
 [111,
  11,
  3,
  721,
  127,
  8,
  63,
  6,
  48,
  11,
  199,
  57,
  11,
  417,
  2,
  23,
  2,
  1616,
  96,
  3,
  12,
  5,
  312,
  1038,
  226,
  2,
  303,
  1146,
  588,
  2976,
  4686,
  608,
  9,
  6,
  4,
  1388,
  10,
  16,
  1473,
  30,
  35,
  1211,
  241,
  24,
  45,
  552],
 [4,
  463,
  5,
  326,
  533,
  8,
  2528,
  620,
  12,
  5,
  32,
  26,
  367,
  184,
  2,
  230,
  14,
  155,
  28,
  3365,
  31,
  44,
  20,
  9,
  6,
  4,
  313,
  463,
  16,
  421,
  101,
  38,
  827,
  3589,
  13,
  4,
  3818,
  4871],
 [197,
  6,
  4,
  2181,
  18,
  4005,
  3447,
  16,
  14099,
  3233,
  907,
  111,
  31,
  14100,
  14101,
  8,
  6,
  4,
  287,
  2,
  708,
  10,
  7,
  1268,
  19,
  5,
  386,
  268,
  287,
  407,
  14,
  2,
  22,
  194,
  9,
  2134,
  3,
  168,
 

In [202]:
# Convert into sequences of integers

X_train = tokenizer.texts_to_sequences(X_train)
X_valid= tokenizer.texts_to_sequences(X_valid)
X_test = tokenizer.texts_to_sequences(X_test)

In [203]:
# Define values important for padding

max_length = 100
trunc_type = "post"
padding_type = "post"

In [204]:
# Pad train, validation and test data

X_train = pad_sequences(X_train, padding=padding_type, 
                        maxlen=max_length, truncating=trunc_type)

X_valid = pad_sequences(X_valid, padding=padding_type, 
                        maxlen=max_length, truncating=trunc_type)

X_test = pad_sequences(X_test, padding=padding_type, 
                       maxlen=max_length, truncating=trunc_type)

In [205]:
# Define vocabulary size

vocab_size = len(tokenizer.word_index) + 1

In [206]:
# Load pretrained embeddings

embedding_vector = dict()

# https://edlitera-datasets.s3.amazonaws.com/glove.6B.100d.txt
word_embeddings = "glove.6B.100d.txt"

f = open(word_embeddings, encoding="utf8")

for line in f:
    values = line.split()
    word = values[0]
    coef = np.asarray(values[1:], dtype="float32")
    embedding_vector[word] = coef
    
f.close()

In [80]:
# Create an embedding matrix to use as weights for the embedding layer

embedding_matrix = np.zeros((vocab_size,100))

for word,i in tokenizer.word_index.items():
    
    embedding_value = embedding_vector.get(word)
    
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

In [81]:
# Define embedding layer with pretrained weights

embedding_layer = Embedding(vocab_size, 
                            100, 
                            weights=[embedding_matrix], 
                            input_length=100, 
                            trainable=False)

In [82]:
# Define model

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(16, recurrent_dropout=0.4)) 
model.add(Dropout(0.5))
#model.add(Dense(4, activation="sigmoid"))
model.add(Dense(1,activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1782000   
                                                                 
 lstm (LSTM)                 (None, 16)                7488      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1,789,505
Trainable params: 7,505
Non-trainable params: 1,782,000
_________________________________________________________________


In [83]:
# Compile model

model.compile(optimizer=Adam(),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

In [84]:
# Define training parameters

num_epochs = 10
batch_size = 128

# Train model

history = model.fit(X_train, 
                    y_train, 
                    batch_size=batch_size, 
                    epochs=num_epochs, 
                    verbose=1, 
                    validation_data=(X_valid, y_valid))

Epoch 1/10
169/169 [==============================] - 10s 42ms/step - loss: 0.6472 - binary_accuracy: 0.6271 - val_loss: 0.5922 - val_binary_accuracy: 0.6962
Epoch 2/10
169/169 [==============================] - 7s 43ms/step - loss: 0.5929 - binary_accuracy: 0.6905 - val_loss: 0.5168 - val_binary_accuracy: 0.7493
Epoch 3/10
169/169 [==============================] - 7s 40ms/step - loss: 0.5308 - binary_accuracy: 0.7440 - val_loss: 0.4757 - val_binary_accuracy: 0.7748
Epoch 4/10
169/169 [==============================] - 7s 43ms/step - loss: 0.4944 - binary_accuracy: 0.7692 - val_loss: 0.4706 - val_binary_accuracy: 0.7744
Epoch 5/10
169/169 [==============================] - 7s 42ms/step - loss: 0.4792 - binary_accuracy: 0.7762 - val_loss: 0.4521 - val_binary_accuracy: 0.7912
Epoch 6/10
169/169 [==============================] - 7s 43ms/step - loss: 0.4675 - binary_accuracy: 0.7850 - val_loss: 0.4681 - val_binary_accuracy: 0.7780
Epoch 7/10
169/169 [==============================] - 7s 

In [85]:
# Make predictions

y_pred = model.predict(X_test)

y_pred = y_pred > 0.6

In [86]:
# Create classification report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.80      0.80      3332
           1       0.81      0.80      0.80      3395

    accuracy                           0.80      6727
   macro avg       0.80      0.80      0.80      6727
weighted avg       0.80      0.80      0.80      6727



## `Keras with pre-trained GloVe embeddings` take home exercise

**Load the dataset available at `https://edlitera-datasets.s3.amazonaws.com/womens_clothing_reviews.csv` and create a DataFrame from it. Using the 
data inside the `Review Text` column, try to predict the appropriate value for the `Recommended IND` column.**

**Note 1: perform undersampling to balance out your dataset**

**Note 2: since we haven't yet covered LSTMs you can use the model we used in class**

## `Solution:`

In [277]:
df = pd.read_csv('https://edlitera-datasets.s3.amazonaws.com/womens_clothing_reviews.csv')

In [278]:
df['Review Text'] = [x.split('.')[0].split() for x in df['Review Text']]

In [279]:
df['Review Text']

0        [I, had, such, high, hopes, for, this, dress, ...
1                  [I, love,, love,, love, this, jumpsuit]
2        [This, shirt, is, very, flattering, to, all, d...
3        [I, love, tracy, reese, dresses,, but, this, o...
4        [I, aded, this, in, my, basket, at, hte, last,...
                               ...                        
19657    [I, was, very, happy, to, snag, this, dress, a...
19658            [It, reminds, me, of, maternity, clothes]
19659    [This, fit, well,, but, the, top, was, very, s...
19660    [I, bought, this, dress, for, a, wedding, i, h...
19661    [This, dress, in, a, lovely, platinum, is, fem...
Name: Review Text, Length: 19662, dtype: object

In [280]:
df['Recommended IND'].value_counts()

1    16087
0     3575
Name: Recommended IND, dtype: int64

In [281]:
count_class_1, count_class_0 = df['Recommended IND'].value_counts()
print(count_class_1, count_class_0)

class_0 = df[df['Recommended IND'] == 0]
class_1 = df[df['Recommended IND'] == 1]
count_class_1_under = class_1.sample(count_class_0)


print(count_class_0)
print(count_class_1)
print(len(count_class_1_under))




16087 3575
3575
16087
3575


In [282]:
df1 = pd.concat([class_0,count_class_1_under],axis = 0)

In [283]:
df1 = df1.sample(frac=1).reset_index(drop=True)

In [284]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer

In [285]:
# def preprocess_reviews(text, stop_words=None, stemmer=None):
#     text_data = text.lower() # lowercase text data
#     tokens = tokenizer.tokenize(text_data) # tokenize data using the given tokenizer
#     if stop_words is not None:
#         tokens = [t for t in tokens if not t in stop_words] # remove stop words
#     if stemmer is not None:
#         tokens = [stemmer.stem(t) for t in tokens] # stem text data
#     return " ".join(tokens)

In [286]:
X = df1['Review Text']



# stemmer = SnowballStemmer(language="english")
# stop_words = set(stopwords.words("english"))

# df1["Review Text"] = df1["Review Text"].apply(X, args=(stop_words, stemmer))


# df1["Review Text"] = df1["Review Text"].map(preprocess_reviews)

Y = df1['Recommended IND']

In [287]:
print(X,Y)

0       [This, runs, very, small, and, has, a, beautif...
1                                            [Great, fit]
2       [I'm, petite,, but, even, if, i, weren't,, i, ...
3       [This, dress, is, amazing!, i, have, to, say, ...
4       [Took, a, chance, on, this, blouse, and, so, g...
                              ...                        
7145    [Needed, a, pair, of, grey, pants, to, go, wit...
7146                                 [I, got, the, ivory]
7147    [I, just, got, this, in, and, i, love, it!, it...
7148    [I, didn't, mind, the, "scratchiness", others,...
7149    [I, wanted, to, love, this, dress, so, much, s...
Name: Review Text, Length: 7150, dtype: object 0       0
1       1
2       0
3       1
4       1
       ..
7145    1
7146    1
7147    1
7148    0
7149    0
Name: Recommended IND, Length: 7150, dtype: int64


In [288]:
# from nltk.corpus import stopwords
# stop_words = stopwords.words("english")

In [289]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2,random_state = 44)
x_valid,x_test,y_valid,y_test = train_test_split(x_test,y_test,test_size = 0.2,random_state = 44)

In [ ]:
tokenizer = Tokenizer(
    num_words=10_000, 
    oov_token = "<OOV>")



In [290]:
tokenizer.fit_on_texts(x_train)

In [291]:
x_train = tokenizer.texts_to_sequences(x_train)
x_valid= tokenizer.texts_to_sequences(x_valid)
x_test = tokenizer.texts_to_sequences(x_test)

In [292]:
x_train

[[3, 99, 7, 16, 10, 2, 5231, 27, 47, 4, 5, 107, 28, 212],
 [3, 12, 1672, 14, 2, 1155, 153, 53, 3, 8340, 403, 9, 161, 276, 417],
 [7, 102, 8, 508, 4, 43, 89],
 [7,
  16,
  8,
  145,
  307,
  4,
  43,
  1975,
  3,
  12,
  791,
  22,
  2,
  1954,
  590,
  281,
  20,
  32,
  1266,
  11,
  50,
  84,
  202,
  28,
  48,
  342,
  40,
  66],
 [95,
  30,
  1299,
  2,
  29,
  8,
  1266,
  11,
  31,
  672,
  31,
  45,
  1839,
  186,
  2670,
  2,
  767,
  57,
  4369],
 [3, 203, 7, 23, 9, 32, 225, 70],
 [3,
  71,
  7,
  13,
  166,
  614,
  5,
  12,
  6,
  42,
  8341,
  11,
  53,
  5,
  328,
  10,
  3,
  203,
  5,
  9,
  29,
  25,
  6,
  519,
  293,
  741],
 [7,
  16,
  12,
  20,
  151,
  3,
  54,
  8342,
  104,
  2,
  106,
  3366,
  3,
  54,
  165,
  5232,
  11,
  2,
  29,
  12,
  43,
  3241,
  4,
  2,
  69,
  452,
  303],
 [7, 57, 535, 15, 196, 150, 63, 4, 150, 9, 1868, 4, 3, 109, 41, 5, 18, 294],
 [44, 7, 10, 8343, 3, 130, 38, 325, 399, 4, 336],
 [2,
  23,
  8,
  1869,
  3,
  41,
  60,
  126,
  4,

In [293]:
# Pad train, validation and test data

x_train = pad_sequences(x_train, padding=padding_type, 
                        maxlen=max_length, truncating=trunc_type)

x_valid = pad_sequences(x_valid, padding=padding_type, 
                        maxlen=max_length, truncating=trunc_type)

x_test = pad_sequences(x_test, padding=padding_type, 
                       maxlen=max_length, truncating=trunc_type)

In [294]:
#print("Count of characters:",tokenizer.word_counts)
#print("Length of text:",tokenizer.document_count)
#print("Character index",tokenizer.word_index)
print("Frequency of characters:",tokenizer.word_docs)

Frequency of characters: defaultdict(<class 'int'>, {'belly': 50, 'will': 672, "i'm": 1178, 'my': 2203, 'scratchy': 58, 'rough': 19, 'very': 1833, 'barely': 54, 'material': 720, 'this': 7019, 'all': 736, 'not': 2160, 'at': 1175, 'be': 1502, 'and': 6359, 'the': 6880, 'short': 478, 'button': 116, 'cut': 472, 'like': 1772, 'returning': 172, 'waisted': 68, 'did': 479, 'is': 4615, 'strangely': 10, 'covered': 27, 'heirloom': 1, 'favorite': 141, 'top': 1861, 'perfect': 664, 'coverage': 32, 'would': 1243, 'if': 888, 'special': 64, 'an': 826, 'photo': 199, 'i': 7102, 'love': 2230, 'much': 751, 'cozy': 84, 'beautiful': 760, 'down': 567, 'could': 476, 'wear': 1206, 'gown': 10, 'up': 1003, 'picks': 6, 'it': 4992, 'everyday': 21, 'soft': 672, 'so': 2539, 'skirt': 443, 'prettier': 31, 'feels': 137, 'color': 1079, 'than': 714, 'hands': 17, 'vibrant': 77, 'green': 168, 'sweater': 643, 'dress': 2261, 'navy': 109, 'size': 1555, 'with': 2324, 'pjs': 5, 'rich': 42, 'while': 247, 'or': 1014, 'boots': 112, 

In [295]:
vocab_size = len(tokenizer.word_index) + 1


In [296]:

embedding_matrix = np.zeros((vocab_size,100))

for word,i in tokenizer.word_index.items():
    
    embedding_value = embedding_vector.get(word)
    
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

In [297]:
# Define embedding layer with pretrained weights

embedding_layer = Embedding(vocab_size, 
                            100, 
                            weights=[embedding_matrix], 
                            input_length=100, 
                            trainable=False)

In [320]:
# Define model

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(32, recurrent_dropout=0.4)) 
#model.add(LSTM(16, recurrent_dropout=0.4)) 
#model.add(LSTM(8, recurrent_dropout=0.4))
model.add(Dropout(0.5))
#model.add(Dense(4, activation="sigmoid"))
model.add(Dense(1,activation="sigmoid"))

model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 100)          1032800   
                                                                 
 lstm_23 (LSTM)              (None, 32)                17024     
                                                                 
 dropout_9 (Dropout)         (None, 32)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,049,857
Trainable params: 17,057
Non-trainable params: 1,032,800
_________________________________________________________________


In [321]:
# Compile model

model.compile(optimizer=Adam(),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

In [322]:
# Define training parameters

num_epochs = 500
batch_size = 128

# Train model

history = model.fit(x_train, 
                    y_train, 
                    batch_size=batch_size, 
                    epochs=num_epochs, 
                    verbose=1, 
                    validation_data=(x_valid, y_valid))

Epoch 1/500
45/45 [==============================] - 5s 64ms/step - loss: 0.6932 - binary_accuracy: 0.4977 - val_loss: 0.6931 - val_binary_accuracy: 0.5149
Epoch 2/500
45/45 [==============================] - 2s 55ms/step - loss: 0.6931 - binary_accuracy: 0.4977 - val_loss: 0.6934 - val_binary_accuracy: 0.4851
Epoch 3/500
45/45 [==============================] - 3s 56ms/step - loss: 0.6935 - binary_accuracy: 0.5023 - val_loss: 0.6930 - val_binary_accuracy: 0.5149
Epoch 4/500
45/45 [==============================] - 2s 54ms/step - loss: 0.6932 - binary_accuracy: 0.4958 - val_loss: 0.6931 - val_binary_accuracy: 0.5149
Epoch 5/500
45/45 [==============================] - 3s 57ms/step - loss: 0.6931 - binary_accuracy: 0.4949 - val_loss: 0.6934 - val_binary_accuracy: 0.4851
Epoch 6/500
45/45 [==============================] - 2s 54ms/step - loss: 0.6931 - binary_accuracy: 0.5017 - val_loss: 0.6930 - val_binary_accuracy: 0.5149
Epoch 7/500
45/45 [==============================] - 2s 55ms/ste

45/45 [==============================] - 3s 72ms/step - loss: 0.4453 - binary_accuracy: 0.7970 - val_loss: 0.6646 - val_binary_accuracy: 0.6976
Epoch 106/500
45/45 [==============================] - 3s 76ms/step - loss: 0.4337 - binary_accuracy: 0.8035 - val_loss: 0.6673 - val_binary_accuracy: 0.6914
Epoch 107/500
45/45 [==============================] - 3s 72ms/step - loss: 0.4314 - binary_accuracy: 0.8075 - val_loss: 0.7204 - val_binary_accuracy: 0.6731
Epoch 108/500
45/45 [==============================] - 3s 69ms/step - loss: 0.4290 - binary_accuracy: 0.8035 - val_loss: 0.6503 - val_binary_accuracy: 0.6897
Epoch 109/500
45/45 [==============================] - 4s 81ms/step - loss: 0.4298 - binary_accuracy: 0.8065 - val_loss: 0.6708 - val_binary_accuracy: 0.7072
Epoch 110/500
45/45 [==============================] - 3s 76ms/step - loss: 0.4275 - binary_accuracy: 0.8021 - val_loss: 0.7042 - val_binary_accuracy: 0.6836
Epoch 111/500
45/45 [==============================] - 3s 75ms/ste

Epoch 157/500
45/45 [==============================] - 3s 63ms/step - loss: 0.3216 - binary_accuracy: 0.8691 - val_loss: 0.7595 - val_binary_accuracy: 0.7028
Epoch 158/500
45/45 [==============================] - 3s 63ms/step - loss: 0.3149 - binary_accuracy: 0.8727 - val_loss: 0.7841 - val_binary_accuracy: 0.7019
Epoch 159/500
45/45 [==============================] - 3s 63ms/step - loss: 0.3219 - binary_accuracy: 0.8680 - val_loss: 0.8140 - val_binary_accuracy: 0.6853
Epoch 160/500
45/45 [==============================] - 3s 64ms/step - loss: 0.3368 - binary_accuracy: 0.8664 - val_loss: 0.7701 - val_binary_accuracy: 0.7098
Epoch 161/500
45/45 [==============================] - 3s 67ms/step - loss: 0.3191 - binary_accuracy: 0.8726 - val_loss: 0.7981 - val_binary_accuracy: 0.6984
Epoch 162/500
45/45 [==============================] - 3s 65ms/step - loss: 0.3138 - binary_accuracy: 0.8743 - val_loss: 0.8126 - val_binary_accuracy: 0.6993
Epoch 163/500
45/45 [==============================]

Epoch 209/500
45/45 [==============================] - 3s 62ms/step - loss: 0.2530 - binary_accuracy: 0.9042 - val_loss: 0.8655 - val_binary_accuracy: 0.7037
Epoch 210/500
45/45 [==============================] - 3s 60ms/step - loss: 0.2481 - binary_accuracy: 0.9124 - val_loss: 0.9273 - val_binary_accuracy: 0.7045
Epoch 211/500
45/45 [==============================] - 3s 61ms/step - loss: 0.2364 - binary_accuracy: 0.9164 - val_loss: 0.8974 - val_binary_accuracy: 0.7019
Epoch 212/500
45/45 [==============================] - 3s 63ms/step - loss: 0.2452 - binary_accuracy: 0.9094 - val_loss: 0.9691 - val_binary_accuracy: 0.7037
Epoch 213/500
45/45 [==============================] - 3s 63ms/step - loss: 0.2551 - binary_accuracy: 0.9054 - val_loss: 0.9242 - val_binary_accuracy: 0.7010
Epoch 214/500
45/45 [==============================] - 3s 63ms/step - loss: 0.2521 - binary_accuracy: 0.9061 - val_loss: 0.9086 - val_binary_accuracy: 0.6914
Epoch 215/500
45/45 [==============================]

Epoch 261/500
45/45 [==============================] - 3s 67ms/step - loss: 0.2025 - binary_accuracy: 0.9323 - val_loss: 1.1669 - val_binary_accuracy: 0.6818
Epoch 262/500
45/45 [==============================] - 3s 63ms/step - loss: 0.1998 - binary_accuracy: 0.9315 - val_loss: 1.0518 - val_binary_accuracy: 0.6923
Epoch 263/500
45/45 [==============================] - 3s 68ms/step - loss: 0.1988 - binary_accuracy: 0.9322 - val_loss: 1.0764 - val_binary_accuracy: 0.7028
Epoch 264/500
45/45 [==============================] - 3s 62ms/step - loss: 0.1927 - binary_accuracy: 0.9334 - val_loss: 1.0203 - val_binary_accuracy: 0.6984
Epoch 265/500
45/45 [==============================] - 3s 63ms/step - loss: 0.1845 - binary_accuracy: 0.9390 - val_loss: 1.0476 - val_binary_accuracy: 0.7115
Epoch 266/500
45/45 [==============================] - 3s 60ms/step - loss: 0.2014 - binary_accuracy: 0.9320 - val_loss: 1.0023 - val_binary_accuracy: 0.7010
Epoch 267/500
45/45 [==============================]

Epoch 313/500
45/45 [==============================] - 3s 63ms/step - loss: 0.1676 - binary_accuracy: 0.9474 - val_loss: 1.2068 - val_binary_accuracy: 0.6888
Epoch 314/500
45/45 [==============================] - 3s 62ms/step - loss: 0.1655 - binary_accuracy: 0.9491 - val_loss: 1.1499 - val_binary_accuracy: 0.6914
Epoch 315/500
45/45 [==============================] - 3s 63ms/step - loss: 0.1561 - binary_accuracy: 0.9509 - val_loss: 1.1626 - val_binary_accuracy: 0.6976
Epoch 316/500
45/45 [==============================] - 3s 63ms/step - loss: 0.1593 - binary_accuracy: 0.9512 - val_loss: 1.1291 - val_binary_accuracy: 0.6958
Epoch 317/500
45/45 [==============================] - 3s 64ms/step - loss: 0.1516 - binary_accuracy: 0.9559 - val_loss: 1.1844 - val_binary_accuracy: 0.6949
Epoch 318/500
45/45 [==============================] - 3s 63ms/step - loss: 0.1620 - binary_accuracy: 0.9517 - val_loss: 1.1917 - val_binary_accuracy: 0.6984
Epoch 319/500
45/45 [==============================]

Epoch 365/500
45/45 [==============================] - 3s 65ms/step - loss: 0.1466 - binary_accuracy: 0.9554 - val_loss: 1.2758 - val_binary_accuracy: 0.6879
Epoch 366/500
45/45 [==============================] - 3s 61ms/step - loss: 0.1461 - binary_accuracy: 0.9561 - val_loss: 1.2338 - val_binary_accuracy: 0.6853
Epoch 367/500
45/45 [==============================] - 3s 63ms/step - loss: 0.1518 - binary_accuracy: 0.9507 - val_loss: 1.2052 - val_binary_accuracy: 0.6906
Epoch 368/500
45/45 [==============================] - 3s 63ms/step - loss: 0.1317 - binary_accuracy: 0.9608 - val_loss: 1.3020 - val_binary_accuracy: 0.6914
Epoch 369/500
45/45 [==============================] - 3s 62ms/step - loss: 0.1443 - binary_accuracy: 0.9580 - val_loss: 1.2550 - val_binary_accuracy: 0.6906
Epoch 370/500
45/45 [==============================] - 3s 64ms/step - loss: 0.1462 - binary_accuracy: 0.9551 - val_loss: 1.2368 - val_binary_accuracy: 0.6949
Epoch 371/500
45/45 [==============================]

Epoch 417/500
45/45 [==============================] - 4s 78ms/step - loss: 0.1309 - binary_accuracy: 0.9635 - val_loss: 1.3558 - val_binary_accuracy: 0.6862
Epoch 418/500
45/45 [==============================] - 3s 66ms/step - loss: 0.1221 - binary_accuracy: 0.9664 - val_loss: 1.3985 - val_binary_accuracy: 0.6862
Epoch 419/500
45/45 [==============================] - 3s 63ms/step - loss: 0.1246 - binary_accuracy: 0.9650 - val_loss: 1.4365 - val_binary_accuracy: 0.6914
Epoch 420/500
45/45 [==============================] - 3s 67ms/step - loss: 0.1211 - binary_accuracy: 0.9685 - val_loss: 1.3854 - val_binary_accuracy: 0.6949
Epoch 421/500
45/45 [==============================] - 3s 65ms/step - loss: 0.1257 - binary_accuracy: 0.9663 - val_loss: 1.3725 - val_binary_accuracy: 0.6888
Epoch 422/500
45/45 [==============================] - 3s 66ms/step - loss: 0.1147 - binary_accuracy: 0.9703 - val_loss: 1.3623 - val_binary_accuracy: 0.6888
Epoch 423/500
45/45 [==============================]

Epoch 469/500
45/45 [==============================] - 3s 66ms/step - loss: 0.1034 - binary_accuracy: 0.9743 - val_loss: 1.4580 - val_binary_accuracy: 0.6853
Epoch 470/500
45/45 [==============================] - 3s 61ms/step - loss: 0.1098 - binary_accuracy: 0.9708 - val_loss: 1.4038 - val_binary_accuracy: 0.6932
Epoch 471/500
45/45 [==============================] - 3s 63ms/step - loss: 0.1082 - binary_accuracy: 0.9715 - val_loss: 1.4375 - val_binary_accuracy: 0.6766
Epoch 472/500
45/45 [==============================] - 3s 61ms/step - loss: 0.1101 - binary_accuracy: 0.9722 - val_loss: 1.4288 - val_binary_accuracy: 0.6748
Epoch 473/500
45/45 [==============================] - 3s 64ms/step - loss: 0.1036 - binary_accuracy: 0.9736 - val_loss: 1.4696 - val_binary_accuracy: 0.6836
Epoch 474/500
45/45 [==============================] - 3s 64ms/step - loss: 0.1143 - binary_accuracy: 0.9719 - val_loss: 1.4103 - val_binary_accuracy: 0.6774
Epoch 475/500
45/45 [==============================]

In [323]:
y_pred = model.predict(x_test)


y_pred = y_pred > 0.6

In [324]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.58      0.63       158
           1       0.57      0.67      0.61       128

    accuracy                           0.62       286
   macro avg       0.63      0.63      0.62       286
weighted avg       0.63      0.62      0.62       286



# Solution from teacher

In [325]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dense, LSTM, Dropout
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.metrics import BinaryAccuracy
from numpy.random import seed
seed(42) 
import tensorflow as tf
tf.random.set_seed(42)

df = pd.read_csv("https://edlitera-datasets.s3.amazonaws.com/womens_clothing_reviews.csv")
df_class_0 = df[df["Recommended IND"] == 0]
df_class_1 = df[df["Recommended IND"] == 1]

df_class_1 = df_class_1.sample(len(df_class_0))
df = pd.concat([df_class_0, df_class_1])



df = df.sample(frac=1).reset_index(drop=True)

X = df["Review Text"]

y = df["Recommended IND"]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)


X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.20, random_state=42
)

# Define tokenizer
# Set number of words as 10 000
# Set value of oov token
# Leave everything else on default values

tokenizer = Tokenizer(
    num_words=10_000, 
    oov_token = "<OOV>")

# Fit tokenizer on train data

tokenizer.fit_on_texts(X_train)

# Convert into sequences of integers

X_train = tokenizer.texts_to_sequences(X_train)
X_valid = tokenizer.texts_to_sequences(X_valid)
X_test = tokenizer.texts_to_sequences(X_test)

# Define values important for padding

max_length = 100
trunc_type = "post"
padding_type = "post"

# Pad train, validation and test data

X_train = pad_sequences(X_train, padding=padding_type, maxlen=max_length, truncating=trunc_type)
X_valid = pad_sequences(X_valid, padding=padding_type, maxlen=max_length, truncating=trunc_type)
X_test = pad_sequences(X_test, padding=padding_type, maxlen=max_length, truncating=trunc_type)

# Define vocabulary size

vocab_size = len(tokenizer.word_index)+1

# Load pretrained embeddings

embedding_vector = dict()

word_embeddings = "glove.6B.100d.txt"

f = open(word_embeddings, encoding="utf8")

for line in f:
    values = line.split()
    word = values[0]
    coef = np.asarray(values[1:], dtype="float32")
    embedding_vector[word] = coef
    
f.close()


# Create an embedding matrix to use as weights for the embedding layer

embedding_matrix = np.zeros((vocab_size,100))
for word,i in tokenizer.word_index.items():
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value
        
# Define embedding layer with pretrained weights

embedding_layer = Embedding(vocab_size, 
                            100, 
                            weights=[embedding_matrix], 
                            input_length=100, 
                            trainable=False)


# Define model

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(16, recurrent_dropout=0.2)) 
model.add(Dropout(0.5))
model.add(Dense(4, activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.summary()

# Compile model

model.compile(optimizer=Adam(),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])
              
# Define training parameters

num_epochs = 500
batch_size = 64

# Train model

history = model.fit(X_train, 
                    y_train, 
                    batch_size=batch_size, 
                    epochs=num_epochs, 
                    verbose=1, 
                    validation_data=(X_valid, y_valid))
              
# Make predictions

y_pred = model.predict(X_test)

y_pred = y_pred > 0.5
# Create classification report

print(classification_report(y_test, y_pred))

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 100, 100)          769800    
                                                                 
 lstm_24 (LSTM)              (None, 16)                7488      
                                                                 
 dropout_10 (Dropout)        (None, 16)                0         
                                                                 
 dense_12 (Dense)            (None, 4)                 68        
                                                                 
 dense_13 (Dense)            (None, 1)                 5         
                                                                 
Total params: 777,361
Trainable params: 7,561
Non-trainable params: 769,800
_________________________________________________________________
Epoch 1/500
72/72 [========================

72/72 [==============================] - 3s 41ms/step - loss: 0.1945 - binary_accuracy: 0.9388 - val_loss: 0.4470 - val_binary_accuracy: 0.8392
Epoch 99/500
72/72 [==============================] - 3s 37ms/step - loss: 0.1899 - binary_accuracy: 0.9401 - val_loss: 0.4408 - val_binary_accuracy: 0.8400
Epoch 100/500
72/72 [==============================] - 3s 38ms/step - loss: 0.1800 - binary_accuracy: 0.9412 - val_loss: 0.4628 - val_binary_accuracy: 0.8444
Epoch 101/500
72/72 [==============================] - 3s 37ms/step - loss: 0.1918 - binary_accuracy: 0.9386 - val_loss: 0.4326 - val_binary_accuracy: 0.8357
Epoch 102/500
72/72 [==============================] - 3s 36ms/step - loss: 0.1811 - binary_accuracy: 0.9412 - val_loss: 0.4940 - val_binary_accuracy: 0.8383
Epoch 103/500
72/72 [==============================] - 3s 37ms/step - loss: 0.1762 - binary_accuracy: 0.9436 - val_loss: 0.4620 - val_binary_accuracy: 0.8374
Epoch 104/500
72/72 [==============================] - 3s 38ms/step

Epoch 150/500
72/72 [==============================] - 3s 39ms/step - loss: 0.1274 - binary_accuracy: 0.9650 - val_loss: 0.5312 - val_binary_accuracy: 0.8339
Epoch 151/500
72/72 [==============================] - 3s 37ms/step - loss: 0.1218 - binary_accuracy: 0.9681 - val_loss: 0.5623 - val_binary_accuracy: 0.8435
Epoch 152/500
72/72 [==============================] - 3s 38ms/step - loss: 0.1481 - binary_accuracy: 0.9565 - val_loss: 0.5071 - val_binary_accuracy: 0.8357
Epoch 153/500
72/72 [==============================] - 3s 36ms/step - loss: 0.1292 - binary_accuracy: 0.9620 - val_loss: 0.5430 - val_binary_accuracy: 0.8330
Epoch 154/500
72/72 [==============================] - 3s 37ms/step - loss: 0.1238 - binary_accuracy: 0.9679 - val_loss: 0.5477 - val_binary_accuracy: 0.8339
Epoch 155/500
72/72 [==============================] - 3s 36ms/step - loss: 0.1183 - binary_accuracy: 0.9659 - val_loss: 0.5679 - val_binary_accuracy: 0.8400
Epoch 156/500
72/72 [==============================]

Epoch 202/500
72/72 [==============================] - 3s 41ms/step - loss: 0.1040 - binary_accuracy: 0.9736 - val_loss: 0.5857 - val_binary_accuracy: 0.8191
Epoch 203/500
72/72 [==============================] - 3s 40ms/step - loss: 0.1058 - binary_accuracy: 0.9718 - val_loss: 0.6107 - val_binary_accuracy: 0.8295
Epoch 204/500
72/72 [==============================] - 3s 41ms/step - loss: 0.0953 - binary_accuracy: 0.9753 - val_loss: 0.6277 - val_binary_accuracy: 0.8330
Epoch 205/500
72/72 [==============================] - 3s 40ms/step - loss: 0.0967 - binary_accuracy: 0.9742 - val_loss: 0.6177 - val_binary_accuracy: 0.8339
Epoch 206/500
72/72 [==============================] - 3s 40ms/step - loss: 0.0938 - binary_accuracy: 0.9760 - val_loss: 0.6081 - val_binary_accuracy: 0.8357
Epoch 207/500
72/72 [==============================] - 3s 38ms/step - loss: 0.0918 - binary_accuracy: 0.9771 - val_loss: 0.6696 - val_binary_accuracy: 0.8365
Epoch 208/500
72/72 [==============================]

Epoch 254/500
72/72 [==============================] - 3s 47ms/step - loss: 0.0804 - binary_accuracy: 0.9836 - val_loss: 0.7457 - val_binary_accuracy: 0.8339
Epoch 255/500
72/72 [==============================] - 4s 51ms/step - loss: 0.0836 - binary_accuracy: 0.9788 - val_loss: 0.7424 - val_binary_accuracy: 0.8234
Epoch 256/500
72/72 [==============================] - 4s 49ms/step - loss: 0.0843 - binary_accuracy: 0.9777 - val_loss: 0.6250 - val_binary_accuracy: 0.8383
Epoch 257/500
72/72 [==============================] - 4s 49ms/step - loss: 0.0794 - binary_accuracy: 0.9819 - val_loss: 0.7045 - val_binary_accuracy: 0.8330
Epoch 258/500
72/72 [==============================] - 4s 50ms/step - loss: 0.0741 - binary_accuracy: 0.9845 - val_loss: 0.7280 - val_binary_accuracy: 0.8234
Epoch 259/500
72/72 [==============================] - 4s 54ms/step - loss: 0.1008 - binary_accuracy: 0.9698 - val_loss: 0.6503 - val_binary_accuracy: 0.8383
Epoch 260/500
72/72 [==============================]

Epoch 306/500
72/72 [==============================] - 4s 49ms/step - loss: 0.0815 - binary_accuracy: 0.9806 - val_loss: 0.6848 - val_binary_accuracy: 0.8409
Epoch 307/500
72/72 [==============================] - 3s 48ms/step - loss: 0.0801 - binary_accuracy: 0.9810 - val_loss: 0.6675 - val_binary_accuracy: 0.8295
Epoch 308/500
72/72 [==============================] - 4s 50ms/step - loss: 0.0776 - binary_accuracy: 0.9823 - val_loss: 0.7270 - val_binary_accuracy: 0.8339
Epoch 309/500
72/72 [==============================] - 3s 48ms/step - loss: 0.0753 - binary_accuracy: 0.9834 - val_loss: 0.6868 - val_binary_accuracy: 0.8339
Epoch 310/500
72/72 [==============================] - 3s 46ms/step - loss: 0.0892 - binary_accuracy: 0.9784 - val_loss: 0.6742 - val_binary_accuracy: 0.8348
Epoch 311/500
72/72 [==============================] - 3s 45ms/step - loss: 0.0730 - binary_accuracy: 0.9847 - val_loss: 0.6621 - val_binary_accuracy: 0.8409
Epoch 312/500
72/72 [==============================]

Epoch 358/500
72/72 [==============================] - 4s 49ms/step - loss: 0.0709 - binary_accuracy: 0.9840 - val_loss: 0.7756 - val_binary_accuracy: 0.8383
Epoch 359/500
72/72 [==============================] - 3s 46ms/step - loss: 0.0679 - binary_accuracy: 0.9851 - val_loss: 0.7196 - val_binary_accuracy: 0.8435
Epoch 360/500
72/72 [==============================] - 3s 43ms/step - loss: 0.1135 - binary_accuracy: 0.9698 - val_loss: 0.6279 - val_binary_accuracy: 0.8392
Epoch 361/500
72/72 [==============================] - 3s 45ms/step - loss: 0.0802 - binary_accuracy: 0.9801 - val_loss: 0.7097 - val_binary_accuracy: 0.8462
Epoch 362/500
72/72 [==============================] - 3s 48ms/step - loss: 0.0737 - binary_accuracy: 0.9830 - val_loss: 0.7065 - val_binary_accuracy: 0.8488
Epoch 363/500
72/72 [==============================] - 4s 50ms/step - loss: 0.0691 - binary_accuracy: 0.9843 - val_loss: 0.6917 - val_binary_accuracy: 0.8365
Epoch 364/500
72/72 [==============================]

Epoch 410/500
72/72 [==============================] - 3s 38ms/step - loss: 0.0665 - binary_accuracy: 0.9860 - val_loss: 0.7519 - val_binary_accuracy: 0.8287
Epoch 411/500
72/72 [==============================] - 3s 38ms/step - loss: 0.0702 - binary_accuracy: 0.9843 - val_loss: 0.7794 - val_binary_accuracy: 0.8348
Epoch 412/500
72/72 [==============================] - 3s 41ms/step - loss: 0.0651 - binary_accuracy: 0.9867 - val_loss: 0.7691 - val_binary_accuracy: 0.8383
Epoch 413/500
72/72 [==============================] - 3s 39ms/step - loss: 0.0618 - binary_accuracy: 0.9880 - val_loss: 0.8008 - val_binary_accuracy: 0.8400
Epoch 414/500
72/72 [==============================] - 3s 44ms/step - loss: 0.0646 - binary_accuracy: 0.9865 - val_loss: 0.7994 - val_binary_accuracy: 0.8260
Epoch 415/500
72/72 [==============================] - 3s 45ms/step - loss: 0.0617 - binary_accuracy: 0.9865 - val_loss: 0.7981 - val_binary_accuracy: 0.8400
Epoch 416/500
72/72 [==============================]

Epoch 462/500
72/72 [==============================] - 3s 47ms/step - loss: 0.0735 - binary_accuracy: 0.9832 - val_loss: 0.7956 - val_binary_accuracy: 0.8260
Epoch 463/500
72/72 [==============================] - 3s 47ms/step - loss: 0.0639 - binary_accuracy: 0.9869 - val_loss: 0.7458 - val_binary_accuracy: 0.8287
Epoch 464/500
72/72 [==============================] - 3s 48ms/step - loss: 0.0836 - binary_accuracy: 0.9812 - val_loss: 0.8471 - val_binary_accuracy: 0.8269
Epoch 465/500
72/72 [==============================] - 3s 47ms/step - loss: 0.0678 - binary_accuracy: 0.9834 - val_loss: 0.8530 - val_binary_accuracy: 0.8269
Epoch 466/500
72/72 [==============================] - 3s 45ms/step - loss: 0.0729 - binary_accuracy: 0.9821 - val_loss: 0.8414 - val_binary_accuracy: 0.8287
Epoch 467/500
72/72 [==============================] - 3s 46ms/step - loss: 0.0643 - binary_accuracy: 0.9860 - val_loss: 0.8516 - val_binary_accuracy: 0.8365
Epoch 468/500
72/72 [==============================]

# `spaCy`

* package developed by Matt Honnibal (Explosion AI)

* markets itself as "the package for industrial strength NLP in Python"

## `Characteristics`

* easy to install and use - the user can be a beginner and still leverage its power

* fast - implemented in Cython (superset of Python that compiles to C)

* very customizable - easy to add custom components, supports using custom models in Pytorch, TensorFlow etc.

* robust and rigorously tested - everything is extensively tested before being included into the package (the package gets  updated regularly)

## `Features`

* enables the user to easily automatize common NLP tasks:
    * parsing
    * tokenization
    * lemmatization
    * POS tagging
    * text classification
    * named entity recognition
    * etc.

 <div>
<img src="https://edlitera-images.s3.amazonaws.com/spaCy_features.png" width="1000"/>
</div>

* 63 trained pipelines for 19 languages
    * supports more than 64 languages, but doesn't have already trained pipelines for most of them
    * enables easy creation of custom pipelines and offers tools for workflow management
    * contains pretrained models (such as BERT) and pretrained word vectors

* easy model packaging and deployment

# `Text processing in spaCy`


* to run anything in spaCy we must load a model

* we always first load the model we want to work with
    * **VERY IMPORTANT: pipeline choice is extremely important in spaCy!**
    * to download the english model, open your Python terminal and type in `python -m spacy download en_core_web_lg`
    * you can find out more about the different models spaCy offers on: https://spacy.io/usage/models

* there are a lot of models that we can use to work with text in english

    
    
* main difference between models: **smaller models don't have pretrained word embeddings**
    * since we want to use pretrained embeddings, we used the **en_core_web_lg** model


* let's explain what we can do with a typical model

### `Doc object`


* text processed with a spaCy pipeline will be converted into a **`Doc`** object

* other important concents:
    * **`Span`** objects are slices of the text (point to data inside the **`Doc`** object)
    * **`Token`** objects are elements of the text (also point to data inside the **`Doc`** object)

* we can perform operations on **`Span`** and **`Token`** objects

In [1]:
import spacy

In [2]:
# Define some example text data

text = """Whose woods these are I think I know.
His house is in the village though;
He will not see me stopping here
To watch his woods fill up with snow.
My little horse must think it queer
To stop without a farmhouse near
Between the woods and frozen lake
The darkest evening of the year.
He gives his harness bells a shake
To ask if there is some mistake.
The only other sound’s the sweep
Of easy wind and downy flake.
The woods are lovely, dark and deep,
But I have promises to keep,
And miles to go before I sleep,
And miles to go before I sleep."""

In [3]:
# Define language
# Loads a spaCy model

nlp = spacy.load("en_core_web_lg")

#nlp = spacy.load()

# Create a Doc object

document = nlp(text)

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

**The pipeline automatically performs a number of operations in the background on our text !**

* by accessing different attributes we access what the pipeline already performed in the background

In [39]:
# Show what the pipeline did in the background

nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x1ed5000f360>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x1ed59c98c20>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1ef5a122760>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x1ef5a1a7e40>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x1ef5a19ea00>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1ef59f90dc0>)]

* as can be seen, our data has been:

    * POS tagged
    * parsed
    * lemmatized
    * we can even access named entity recognition attributes
    * word embeddings for each word in the text have been fetched

### `Tokenization and punctuation removal`

* to access a single token we can simply access an index of our **`Doc`** object

In [40]:
# Access a single token

document[1]

woods

* we can also access multiple tokens

In [41]:
# Access multiple tokens

document[1:5]

woods these are I

* since the whole string has been tokeniyed, we can create a list of tokens

In [42]:
# To get token objects we can create a simple loop (or even better, a list comprehension)

tokens = [token for token in document]

In [43]:
tokens

[Whose,
 woods,
 these,
 are,
 I,
 think,
 I,
 know,
 .,
 ,
 His,
 house,
 is,
 in,
 the,
 village,
 though,
 ;,
 ,
 He,
 will,
 not,
 see,
 me,
 stopping,
 here,
 ,
 To,
 watch,
 his,
 woods,
 fill,
 up,
 with,
 snow,
 .,
 ,
 My,
 little,
 horse,
 must,
 think,
 it,
 queer,
 ,
 To,
 stop,
 without,
 a,
 farmhouse,
 near,
 ,
 Between,
 the,
 woods,
 and,
 frozen,
 lake,
 ,
 The,
 darkest,
 evening,
 of,
 the,
 year,
 .,
 ,
 He,
 gives,
 his,
 harness,
 bells,
 a,
 shake,
 ,
 To,
 ask,
 if,
 there,
 is,
 some,
 mistake,
 .,
 ,
 The,
 only,
 other,
 sound,
 ’s,
 the,
 sweep,
 ,
 Of,
 easy,
 wind,
 and,
 downy,
 flake,
 .,
 ,
 The,
 woods,
 are,
 lovely,
 ,,
 dark,
 and,
 deep,
 ,,
 ,
 But,
 I,
 have,
 promises,
 to,
 keep,
 ,,
 ,
 And,
 miles,
 to,
 go,
 before,
 I,
 sleep,
 ,,
 ,
 And,
 miles,
 to,
 go,
 before,
 I,
 sleep,
 .]

* to remove punctuation we can reference the **is_punct** attribute of each token


* we do something similar to remove empty spaces: we access the **is_space** attribute


* to remove stop words we can reference the **is_stop** attribute of some token


* list of token attributes: https://spacy.io/api/token

In [44]:
# Remove punctuation using a list comprehension

tokens_without_punctuation = [token for token in document if not token.is_punct] 

In [45]:
tokens_without_punctuation

[Whose,
 woods,
 these,
 are,
 I,
 think,
 I,
 know,
 ,
 His,
 house,
 is,
 in,
 the,
 village,
 though,
 ,
 He,
 will,
 not,
 see,
 me,
 stopping,
 here,
 ,
 To,
 watch,
 his,
 woods,
 fill,
 up,
 with,
 snow,
 ,
 My,
 little,
 horse,
 must,
 think,
 it,
 queer,
 ,
 To,
 stop,
 without,
 a,
 farmhouse,
 near,
 ,
 Between,
 the,
 woods,
 and,
 frozen,
 lake,
 ,
 The,
 darkest,
 evening,
 of,
 the,
 year,
 ,
 He,
 gives,
 his,
 harness,
 bells,
 a,
 shake,
 ,
 To,
 ask,
 if,
 there,
 is,
 some,
 mistake,
 ,
 The,
 only,
 other,
 sound,
 ’s,
 the,
 sweep,
 ,
 Of,
 easy,
 wind,
 and,
 downy,
 flake,
 ,
 The,
 woods,
 are,
 lovely,
 dark,
 and,
 deep,
 ,
 But,
 I,
 have,
 promises,
 to,
 keep,
 ,
 And,
 miles,
 to,
 go,
 before,
 I,
 sleep,
 ,
 And,
 miles,
 to,
 go,
 before,
 I,
 sleep]

In [46]:
# Remove empty spaces using a list comprehension

tokens_without_empty_spaces = [token for token in document if not token.is_space] 

In [47]:
tokens_without_empty_spaces

[Whose,
 woods,
 these,
 are,
 I,
 think,
 I,
 know,
 .,
 His,
 house,
 is,
 in,
 the,
 village,
 though,
 ;,
 He,
 will,
 not,
 see,
 me,
 stopping,
 here,
 To,
 watch,
 his,
 woods,
 fill,
 up,
 with,
 snow,
 .,
 My,
 little,
 horse,
 must,
 think,
 it,
 queer,
 To,
 stop,
 without,
 a,
 farmhouse,
 near,
 Between,
 the,
 woods,
 and,
 frozen,
 lake,
 The,
 darkest,
 evening,
 of,
 the,
 year,
 .,
 He,
 gives,
 his,
 harness,
 bells,
 a,
 shake,
 To,
 ask,
 if,
 there,
 is,
 some,
 mistake,
 .,
 The,
 only,
 other,
 sound,
 ’s,
 the,
 sweep,
 Of,
 easy,
 wind,
 and,
 downy,
 flake,
 .,
 The,
 woods,
 are,
 lovely,
 ,,
 dark,
 and,
 deep,
 ,,
 But,
 I,
 have,
 promises,
 to,
 keep,
 ,,
 And,
 miles,
 to,
 go,
 before,
 I,
 sleep,
 ,,
 And,
 miles,
 to,
 go,
 before,
 I,
 sleep,
 .]

In [48]:
# Remove stop words using a list comprehension

tokens_no_stopwords = [token for token in document if not token.is_stop] 

In [49]:
tokens_no_stopwords

[woods,
 think,
 know,
 .,
 ,
 house,
 village,
 ;,
 ,
 stopping,
 ,
 watch,
 woods,
 fill,
 snow,
 .,
 ,
 little,
 horse,
 think,
 queer,
 ,
 stop,
 farmhouse,
 near,
 ,
 woods,
 frozen,
 lake,
 ,
 darkest,
 evening,
 year,
 .,
 ,
 gives,
 harness,
 bells,
 shake,
 ,
 ask,
 mistake,
 .,
 ,
 sound,
 sweep,
 ,
 easy,
 wind,
 downy,
 flake,
 .,
 ,
 woods,
 lovely,
 ,,
 dark,
 deep,
 ,,
 ,
 promises,
 ,,
 ,
 miles,
 sleep,
 ,,
 ,
 miles,
 sleep,
 .]

In [50]:
# Remove empty spaces and punctuation using a list comprehension

cleaned_tokens = [token for token in document if not token.is_space | token.is_punct | token.is_stop] 

In [51]:
cleaned_tokens

[woods,
 think,
 know,
 house,
 village,
 stopping,
 watch,
 woods,
 fill,
 snow,
 little,
 horse,
 think,
 queer,
 stop,
 farmhouse,
 near,
 woods,
 frozen,
 lake,
 darkest,
 evening,
 year,
 gives,
 harness,
 bells,
 shake,
 ask,
 mistake,
 sound,
 sweep,
 easy,
 wind,
 downy,
 flake,
 woods,
 lovely,
 dark,
 deep,
 promises,
 miles,
 sleep,
 miles,
 sleep]

**We will often prefer having strings to spaCy tokens when we create Deep Learning models.**

* unless we want to use a model directly from spaCy, we need to convert the tokens back to text data before we e.g. create word embeddings from the preprocessed text


* to get strings we will access the `.orth_ attribute` of spaCy tokens

In [52]:
# Proof that our list members aren't strings
# they are spaCy tokens

type(cleaned_tokens[0])

spacy.tokens.token.Token

* if we want to get a string instead of a token we can access the **orth** attribute of our tokens

In [53]:
# Clean original text and create a list of strings

cleaned_list = [token.orth_ for token in cleaned_tokens] 

In [54]:
cleaned_list

['woods',
 'think',
 'know',
 'house',
 'village',
 'stopping',
 'watch',
 'woods',
 'fill',
 'snow',
 'little',
 'horse',
 'think',
 'queer',
 'stop',
 'farmhouse',
 'near',
 'woods',
 'frozen',
 'lake',
 'darkest',
 'evening',
 'year',
 'gives',
 'harness',
 'bells',
 'shake',
 'ask',
 'mistake',
 'sound',
 'sweep',
 'easy',
 'wind',
 'downy',
 'flake',
 'woods',
 'lovely',
 'dark',
 'deep',
 'promises',
 'miles',
 'sleep',
 'miles',
 'sleep']

In [55]:
type(cleaned_list[0])

str

## `Exercise`

**Clean the following log using spaCy:**


```
05/07/21 04:34:40.938: 05-07 04:34:40.381  1370  1370 E Diag-Router: diag: Feature mask not forwarded, failed to send msg mask for periph wcnss
05/07/21 04:34:40.938: 05-07 04:34:40.381  1370  1370 E Diag-Router: diag: Feature mask not forwarded, failed to send msg mask for periph wdsp
05/07/21 04:34:40.938: 05-07 04:34:40.381  1370  1370 E Diag-Router: diag: Feature mask not forwarded, failed to send msg mask for periph npu
05/07/21 04:34:40.938: 05-07 04:34:40.381  1370  1370 E Diag-Router: diag: Feature mask not forwarded, failed to send msg mask for periph wcnss
05/07/21 04:34:40.938: 05-07 04:34:40.381  1370  1370 E Diag-Router: diag: Feature mask not forwarded, failed to send msg mask for periph wdsp
05/07/21 04:34:40.938: 05-07 04:34:40.381  1370  1370 E Diag-Router: diag: Feature mask not forwarded, failed to send msg mask for periph npu
05/07/21 04:34:41.125: 05-07 04:34:40.588  1471  1493 E QCOMSysDaemon: qdss open file: /sys/bus/coresight/devices/coresight-tmc-etr/curr_sink error: No such file or directory
05/07/21 04:34:41.188: 05-07 04:34:40.642  1471  1493 E QCOMSysDaemon: qdss open file: /sys/bus/coresight/devices/coresight-tmc-etr/curr_sink error: No such file or directory
05/07/21 04:34:41.188: 05-07 04:34:40.665  1493  1493 I coresight-dummy soc: modem_diag: Dummy source enabled
```  

**The result you need to get by cleaning the log is the following list of string:**


```
['diag router diag feature mask forwarded failed send msg mask periph wcnss',
 'diag router diag feature mask forwarded failed send msg mask periph wdsp',
 'diag router diag feature mask forwarded failed send msg mask periph npu',
 'qcomsysdaemon qdss open file bus coresight devices coresight tmc etr error file directory',
 'qcomsysdaemon qdss open file bus coresight devices coresight tmc etr error file directory coresight dummy soc dummy source enabled']
 ```
 
**Hint: you can check whether something is a letter in spaCy using `.is_alpha`**

**BONUS TAKE HOME EXERCISE:** achieve the same result using NLTK and/or TextBlob

## `Solution:`

# `POS tagging in Spacy`

* all of our words also get tagged by the pipeline

* SpaCy uses Penn Treebank POS tags 

In [56]:
# Check the POS tags of our text

pos_tags = [(token, token.tag_) for token in cleaned_tokens]

In [57]:
pos_tags

[(woods, 'NNS'),
 (think, 'VBP'),
 (know, 'VBP'),
 (house, 'NN'),
 (village, 'NN'),
 (stopping, 'VBG'),
 (watch, 'VB'),
 (woods, 'NNS'),
 (fill, 'VB'),
 (snow, 'NN'),
 (little, 'JJ'),
 (horse, 'NN'),
 (think, 'VB'),
 (queer, 'VBP'),
 (stop, 'VB'),
 (farmhouse, 'NN'),
 (near, 'IN'),
 (woods, 'NNS'),
 (frozen, 'JJ'),
 (lake, 'NN'),
 (darkest, 'JJS'),
 (evening, 'NN'),
 (year, 'NN'),
 (gives, 'VBZ'),
 (harness, 'NN'),
 (bells, 'NNS'),
 (shake, 'NN'),
 (ask, 'VB'),
 (mistake, 'NN'),
 (sound, 'NN'),
 (sweep, 'NN'),
 (easy, 'JJ'),
 (wind, 'NN'),
 (downy, 'NN'),
 (flake, 'NN'),
 (woods, 'NNS'),
 (lovely, 'JJ'),
 (dark, 'JJ'),
 (deep, 'JJ'),
 (promises, 'NNS'),
 (miles, 'NNS'),
 (sleep, 'VBP'),
 (miles, 'NNS'),
 (sleep, 'VBP')]

**We can also visualize POS tags !**

* let's visualize POS tags of the following sentence: "There is scarcely any passion without struggle."

In [5]:
new_text = "There is scarcely any passion without struggle."
nlp = spacy.blank("en")
new_doc = nlp(new_text)

In [6]:
# Import what we need to create a visualization of the POS tags

from spacy import displacy
 
displacy.render(new_doc, style="dep")

C:\Users\swaheed\Anaconda3\lib\site-packages\spacy\displacy\__init__.py:133: UserWarning: [W005] Doc object not parsed. This means displaCy won't be able to generate a dependency visualization for it. Make sure the Doc was processed with a model that supports dependency parsing, and not just a language class like `English()`. For more info, see the docs:
https://spacy.io/usage/models
  warnings.warn(Warnings.W005)


# `Lemmatization`

* very simple, just access the lemma_ attribute of some token

In [63]:
# Create list of lemmatized tokens

lemmatized_tokens = [token.lemma_ for token in cleaned_tokens]

In [64]:
lemmatized_tokens

['wood',
 'think',
 'know',
 'house',
 'village',
 'stop',
 'watch',
 'wood',
 'fill',
 'snow',
 'little',
 'horse',
 'think',
 'queer',
 'stop',
 'farmhouse',
 'near',
 'wood',
 'frozen',
 'lake',
 'dark',
 'evening',
 'year',
 'give',
 'harness',
 'bell',
 'shake',
 'ask',
 'mistake',
 'sound',
 'sweep',
 'easy',
 'wind',
 'downy',
 'flake',
 'wood',
 'lovely',
 'dark',
 'deep',
 'promise',
 'mile',
 'sleep',
 'mile',
 'sleep']

* one very useful trick: **accessing the lemma_ attribute will automatically convert our tokens into strings**

In [65]:
type(lemmatized_tokens[0])

str

## `Exercise`

**Create a subset of the first 500 rows of the `https://edlitera-datasets.s3.amazonaws.com/Hotel_Reviews.csv` dataset. Then perform the following processes:**

* remove stopwords
* remove punctuation marks
* lemmatize text data

**Use `spaCy` and `Pandas` to solve the exercise.**

## `Solution:`

# Take home exercise

**In this exercise, you need to preprocess data stored in the `Review Text` column of the modified version of the `Womens clothing reviews` dataset. The dataset is available here: `https://edlitera-datasets.s3.amazonaws.com/womens_clothing_reviews.csv`**

**Preprocessing steps:**

* get rid of all the columns except the "Review Text" and "Class Name" columns
* select the first 1000 rows of the DataFrame
* remove punctuation
* remove stop words
* remove empty spaces
* lemmatize text data 

**Do the preprocessing using `spaCy`. After you finish preprocessing the data in the `Review Text` column, create word vectors for the text using the `Word2Vec CBOW` model and `Gensim`.**

## `Solution:`

0.999915

# `NLTK` vs `Gensim` vs `spaCy`

<img src="https://edlitera-images.s3.amazonaws.com/nltk_vs_spacy_vs_gensim.png" width="1000">

inspired by:
<br>

https://medium.com/activewizards-machine-learning-company/comparison-of-top-6-python-nlp-libraries-c4ce160237eb

 <div>
<img src="https://edlitera-images.s3.amazonaws.com/new_edlitera_logo.png" width="500"/>
</div>